<h1> Importation des bibliothèques </h1>

In [2]:
!pip install 'rasterio'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 78.0 MB/s eta 0:00:00


In [3]:
import os
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.features import rasterize
from rasterio.transform import from_origin
from rasterio.windows import Window



<h1> Création du masque binaire </h1>

In [ ]:

def process_raster_and_shapefile(raster_path, shapefile_path, output_dir):
    """
    Traite un raster et un shapefile pour créer un masque binaire.

    Paramètres:
    -----------
    raster_path : str
        Chemin vers le fichier raster d'entrée
    shapefile_path : str
        Chemin vers le fichier shapefile d'entrée
    output_dir : str
        Répertoire de sauvegarde du masque de sortie
    """
    # Lire le raster pour obtenir ses propriétés
    with rasterio.open(raster_path) as src_raster:
        # Récupérer les métadonnées du raster
        raster_crs = src_raster.crs
        raster_transform = src_raster.transform
        raster_shape = src_raster.shape

    # Lire le shapefile
    shapefile_data = gpd.read_file(shapefile_path)

    # Reprojeter le shapefile pour qu'il corresponde au CRS du raster
    shapefile_data = shapefile_data.to_crs(raster_crs)

    # Créer un masque binaire
    mask = rasterize(
        shapes=shapefile_data.geometry,
        out_shape=raster_shape,
        transform=raster_transform,
        fill=0,  # Valeur de fond
        default_value=1,  # Valeur pour la zone des polygones
        dtype=rasterio.uint8
    )

    # Générer le chemin de sortie
    raster_name = os.path.splitext(os.path.basename(raster_path))[0]
    mask_output_path = os.path.join(output_dir, f"{raster_name}_mask.tif")

    # Sauvegarder le masque
    with rasterio.open(
        mask_output_path,
        'w',
        driver='GTiff',
        height=mask.shape[0],
        width=mask.shape[1],
        count=1,
        dtype=mask.dtype,
        crs=raster_crs,
        transform=raster_transform
    ) as dst:
        dst.write(mask, 1)

    print(f"Masque généré et sauvegardé à : {mask_output_path}")

def process_multiple_rasters_and_shapefiles(raster_dir, shapefile_dir, output_dir):
    """
    Traite plusieurs rasters et shapefiles en correspondance.

    Paramètres:
    -----------
    raster_dir : str
        Répertoire contenant les fichiers raster
    shapefile_dir : str
        Répertoire contenant les fichiers shapefile
    output_dir : str
        Répertoire de sauvegarde des masques
    """
    # Lister les fichiers raster et shapefile
    raster_files = [f for f in os.listdir(raster_dir) if f.endswith('.tif')]
    shapefile_files = [f for f in os.listdir(shapefile_dir) if f.endswith('.shp')]

    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)

    # Traiter chaque shapefile
    for shapefile in shapefile_files:
        shapefile_name = os.path.splitext(shapefile)[0]

        # Trouver tous les rasters correspondants (commençant par le nom du shapefile)
        matching_rasters = [
            f for f in raster_files
            if os.path.splitext(f)[0].startswith(shapefile_name)
        ]

        if not matching_rasters:
            print(f"Aucun raster trouvé pour le shapefile : {shapefile}")
            continue

        # Traiter chaque raster correspondant
        for raster in matching_rasters:
            # Construire les chemins complets
            raster_path = os.path.join(raster_dir, raster)
            shapefile_path = os.path.join(shapefile_dir, shapefile)

            print(f"Traitement : {shapefile_name} avec raster {raster}")
            process_raster_and_shapefile(raster_path, shapefile_path, output_dir)




In [ ]:
# Exemple d'utilisation des fonctions
raster_directory = '/content/drive/MyDrive/Colab Notebooks/datasets/data_hackaton/ImageDrone/'
shapefile_directory = '/content/drive/MyDrive/Colab Notebooks/datasets/data_hackaton/Vecteur/'
output_directory = '/content/drive/MyDrive/Colab Notebooks/datasets/data_hackaton/mask/'

# Traiter tous les rasters et shapefiles correspondants
process_multiple_rasters_and_shapefiles(
    raster_directory,
    shapefile_directory,
    output_directory
)

Aucun raster trouvé pour le shapefile : kedougou.shp
Traitement : SANGALKAM avec raster SANGALKAM.tif
Masque généré et sauvegardé à : /content/drive/MyDrive/Colab Notebooks/datasets/data_hackaton/mask/SANGALKAM_mask.tif
Traitement : BALELE avec raster BALELE.tif
Masque généré et sauvegardé à : /content/drive/MyDrive/Colab Notebooks/datasets/data_hackaton/mask/BALELE_mask.tif


<h1> Génération des données d'entrainement du model </h1>


In [ ]:
def create_directory_structure(base_path):
    """
    Crée la structure de dossiers nécessaire pour le traitement des données.

    Args:
        base_path (str): Chemin de base où créer les dossiers
    """
    os.makedirs(os.path.join(base_path, 'data', 'raster'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'data', 'mask'), exist_ok=True)

def extract_raster_info(raster_path):
    """
    Extrait les informations de base du raster.

    Args:
        raster_path (str): Chemin vers le fichier raster

    Returns:
        dict: Informations du raster
    """
    with rasterio.open(raster_path) as src:
        return {
            'width': src.width,
            'height': src.height,
            'profile': src.profile.copy()
        }

def cut_raster(raster_path, mask_path, output_base_path, window_size=256, overlap=50):
    """
    Découpe un raster et son masque en sous-rasters de taille spécifiée tout en préservant leur géolocalisation.
    Ne conserve que les sous-rasters et masques dont le masque n'est pas entièrement nul.

    Args:
        raster_path (str): Chemin vers le fichier raster
        mask_path (str): Chemin vers le fichier mask
        output_base_path (str): Chemin de base pour les fichiers de sortie
        window_size (int, optional): Taille de la fenêtre de découpage. Défaut à 1000.
        overlap (int, optional): Chevauchement entre les sous-rasters. Défaut à 0.
    """
    # Extraire le nom de base du raster
    base_raster_name = os.path.splitext(os.path.basename(raster_path))[0]

    # Ouvrir le raster et le masque
    with rasterio.open(raster_path) as src_raster, rasterio.open(mask_path) as src_mask:
        # Vérification initiale de la cohérence
        assert src_raster.width == src_mask.width, "Largeur du raster et du masque différente"
        assert src_raster.height == src_mask.height, "Hauteur du raster et du masque différente"

        # Copier le profil du raster et du masque original
        raster_profile = src_raster.profile.copy()
        mask_profile = src_mask.profile.copy()

        # Générer les fenêtres de découpage
        for y in range(0, src_raster.height, window_size - overlap):
            for x in range(0, src_raster.width, window_size - overlap):
                # Ajuster la fenêtre aux dimensions du raster
                #window_width = min(window_size, src_raster.width - x)
                #window_height = min(window_size, src_raster.height - y)
                window_width = window_size
                window_height = window_size
                # Créer une fenêtre de lecture
                if window_size > src_raster.width - x :
                    x = x - (window_size - (window_width-x))
                if window_size > src_raster.height - y :
                    y = y - (window_size - (window_height-y))
                window = Window(x, y, window_width, window_height)

                # Calculer les nouvelles coordonnées géographiques pour le sous-raster
                transform = src_raster.window_transform(window)

                # Lire les données du raster et du masque
                raster_data = src_raster.read(window=window)
                mask_data = src_mask.read(window=window)

                # Vérifier si le masque est complètement nul
                # Utiliser une condition qui fonctionne pour les masques multi-bandes et mono-bandes
                if src_mask.count > 1:
                    # Pour les masques multi-bandes, vérifier que toutes les bandes sont nulles
                    is_mask_empty = all(np.all(mask_data[i] == 0) for i in range(src_mask.count))
                else:
                    # Pour les masques mono-bande
                    is_mask_empty = np.all(mask_data == 0)

                # Ne pas traiter si le masque est vide
                if not is_mask_empty:
                    # Préparer les profils pour les sous-rasters
                    sub_raster_profile = raster_profile.copy()
                    sub_mask_profile = mask_profile.copy()

                    # Mettre à jour les profils avec les nouvelles dimensions et transformation
                    sub_raster_profile.update({
                        'height': window_height,
                        'width': window_width,
                        'transform': transform,
                        'count': src_raster.count,
                        'dtype': src_raster.dtypes[0]
                    })

                    sub_mask_profile.update({
                        'height': window_height,
                        'width': window_width,
                        'transform': transform,
                        'count': 1,
                        'dtype': src_mask.dtypes[0]
                    })

                    # Générer le nom de fichier pour ce sous-raster
                    sub_raster_name = f"{base_raster_name}_{x}_{y}.tif"
                    sub_mask_name = f"{base_raster_name}_{x}_{y}_mask.tif"

                    # Chemins complets de sortie
                    raster_output_path = os.path.join(output_base_path, 'data', 'raster', sub_raster_name)
                    mask_output_path = os.path.join(output_base_path, 'data', 'mask', sub_mask_name)

                    # Créer les répertoires de sortie s'ils n'existent pas
                    os.makedirs(os.path.dirname(raster_output_path), exist_ok=True)
                    os.makedirs(os.path.dirname(mask_output_path), exist_ok=True)

                    # Écrire les sous-rasters avec leur géoréférencement correct
                    with rasterio.open(raster_output_path, 'w', **sub_raster_profile) as dst_raster:
                        dst_raster.write(raster_data)

                    with rasterio.open(mask_output_path, 'w', **sub_mask_profile) as dst_mask:
                        dst_mask.write(mask_data)

def process_drone_images(base_path):
    """
    Traite tous les fichiers raster et leurs masques dans le dossier ImageDrone.

    Args:
        base_path (str): Chemin de base contenant les dossiers ImageDrone et mask
    """
    # Créer la structure de dossiers
    create_directory_structure(base_path)

    # Parcourir les fichiers .tif dans le dossier ImageDrone
    for raster_filename in os.listdir(os.path.join(base_path, 'ImageDrone')):
        if raster_filename.endswith('.tif') and not raster_filename.startswith('kedougou'):
            # Chemin complet du raster
            raster_path = os.path.join(base_path, 'ImageDrone', raster_filename)

            # Nom de base du raster sans extension
            base_raster_name = os.path.splitext(raster_filename)[0]

            # Chercher le masque correspondant
            mask_filename = f"{base_raster_name}_mask.tif"
            mask_path = os.path.join(base_path, 'mask', mask_filename)

            # Vérifier si le masque existe
            if os.path.exists(mask_path):
                # Découper le raster et son masque
                cut_raster(raster_path, mask_path, base_path)
            else:
                print(f"Masque non trouvé pour {raster_filename}")


In [8]:
## 1500 découpages maximum
def create_directory_structure(base_path):
    """
    Crée la structure de dossiers nécessaire pour le traitement des données.

    Args:
        base_path (str): Chemin de base où créer les dossiers
    """
    os.makedirs(os.path.join(base_path, 'data', 'raster'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'data', 'mask'), exist_ok=True)

def extract_raster_info(raster_path):
    """
    Extrait les informations de base du raster.

    Args:
        raster_path (str): Chemin vers le fichier raster

    Returns:
        dict: Informations du raster
    """
    with rasterio.open(raster_path) as src:
        return {
            'width': src.width,
            'height': src.height,
            'profile': src.profile.copy()
        }
def cut_raster(raster_path, mask_path, output_base_path, window_size=256, overlap=50, max_subrasters=750):
    """
    Découpe un raster et son masque en sous-rasters de taille spécifiée tout en préservant leur géolocalisation.
    Ne conserve que les sous-rasters et masques dont le masque n'est pas entièrement nul.
    Arrête les découpages après un nombre maximum de sous-rasters.

    Args:
        raster_path (str): Chemin vers le fichier raster
        mask_path (str): Chemin vers le fichier mask
        output_base_path (str): Chemin de base pour les fichiers de sortie
        window_size (int, optional): Taille de la fenêtre de découpage. Défaut à 256.
        overlap (int, optional): Chevauchement entre les sous-rasters. Défaut à 50.
        max_subrasters (int, optional): Nombre maximum de sous-rasters à créer. Défaut à 1500.
    """
    # Extraire le nom de base du raster
    base_raster_name = os.path.splitext(os.path.basename(raster_path))[0]

    # Ouvrir le raster et le masque
    with rasterio.open(raster_path) as src_raster, rasterio.open(mask_path) as src_mask:
        # Vérification initiale de la cohérence
        assert src_raster.width == src_mask.width, "Largeur du raster et du masque différente"
        assert src_raster.height == src_mask.height, "Hauteur du raster et du masque différente"

        # Copier le profil du raster et du masque original
        raster_profile = src_raster.profile.copy()
        mask_profile = src_mask.profile.copy()

        # Compteur de sous-rasters créés
        subrasters_count = 0

        # Générer les fenêtres de découpage
        for y in range(0, src_raster.height, window_size - overlap):
            for x in range(0, src_raster.width, window_size - overlap):
                # Arrêter si le nombre maximum de sous-rasters est atteint
                if subrasters_count >= max_subrasters:
                    print(f"Nombre maximum de sous-rasters ({max_subrasters}) atteint. Arrêt du découpage.")
                    return

                # Ajuster la fenêtre aux dimensions du raster
                window_width = window_size
                window_height = window_size

                # Ajuster les coordonnées si le découpage dépasse les limites du raster
                if window_size > src_raster.width - x:
                    x = x - (window_size - (window_width - x))
                if window_size > src_raster.height - y:
                    y = y - (window_size - (window_height - y))

                window = Window(x, y, window_width, window_height)

                # Calculer les nouvelles coordonnées géographiques pour le sous-raster
                transform = src_raster.window_transform(window)

                # Lire les données du raster et du masque
                raster_data = src_raster.read(window=window)
                mask_data = src_mask.read(window=window)

                # Vérifier si le masque est complètement nul
                # Utiliser une condition qui fonctionne pour les masques multi-bandes et mono-bandes
                if src_mask.count > 1:
                    # Pour les masques multi-bandes, vérifier que toutes les bandes sont nulles
                    is_mask_empty = all(np.all(mask_data[i] == 0) for i in range(src_mask.count))
                else:
                    # Pour les masques mono-bande
                    is_mask_empty = np.all(mask_data == 0)

                # Ne pas traiter si le masque est vide
                if not is_mask_empty:
                    # Préparer les profils pour les sous-rasters
                    sub_raster_profile = raster_profile.copy()
                    sub_mask_profile = mask_profile.copy()

                    # Mettre à jour les profils avec les nouvelles dimensions et transformation
                    sub_raster_profile.update({
                        'height': window_height,
                        'width': window_width,
                        'transform': transform,
                        'count': src_raster.count,
                        'dtype': src_raster.dtypes[0]
                    })

                    sub_mask_profile.update({
                        'height': window_height,
                        'width': window_width,
                        'transform': transform,
                        'count': 1,
                        'dtype': src_mask.dtypes[0]
                    })

                    # Générer le nom de fichier pour ce sous-raster
                    sub_raster_name = f"{base_raster_name}_{x}_{y}.tif"
                    sub_mask_name = f"{base_raster_name}_{x}_{y}_mask.tif"

                    # Chemins complets de sortie
                    raster_output_path = os.path.join(output_base_path, 'data', 'raster', sub_raster_name)
                    mask_output_path = os.path.join(output_base_path, 'data', 'mask', sub_mask_name)

                    # Créer les répertoires de sortie s'ils n'existent pas
                    os.makedirs(os.path.dirname(raster_output_path), exist_ok=True)
                    os.makedirs(os.path.dirname(mask_output_path), exist_ok=True)

                    # Écrire les sous-rasters avec leur géoréférencement correct
                    with rasterio.open(raster_output_path, 'w', **sub_raster_profile) as dst_raster:
                        dst_raster.write(raster_data)

                    with rasterio.open(mask_output_path, 'w', **sub_mask_profile) as dst_mask:
                        dst_mask.write(mask_data)

                    # Incrémenter le compteur de sous-rasters
                    subrasters_count += 1
def process_drone_images(base_path):
    """
    Traite tous les fichiers raster et leurs masques dans le dossier ImageDrone.

    Args:
        base_path (str): Chemin de base contenant les dossiers ImageDrone et mask
    """
    # Créer la structure de dossiers
    create_directory_structure(base_path)

    # Parcourir les fichiers .tif dans le dossier ImageDrone
    for raster_filename in os.listdir(os.path.join(base_path, 'ImageDrone')):
        if raster_filename.endswith('.tif') and not raster_filename.startswith('kedougou'):
            # Chemin complet du raster
            raster_path = os.path.join(base_path, 'ImageDrone', raster_filename)

            # Nom de base du raster sans extension
            base_raster_name = os.path.splitext(raster_filename)[0]

            # Chercher le masque correspondant
            mask_filename = f"{base_raster_name}_mask.tif"
            mask_path = os.path.join(base_path, 'mask', mask_filename)

            # Vérifier si le masque existe
            if os.path.exists(mask_path):
                # Découper le raster et son masque
                cut_raster(raster_path, mask_path, base_path)
            else:
                print(f"Masque non trouvé pour {raster_filename}")


<h2> Application </h2>

In [9]:
base_path = "/content/drive/MyDrive/Colab Notebooks/datasets/data_hackaton"  # Chemin du dossier de base
process_drone_images(base_path)

Nombre maximum de sous-rasters (750) atteint. Arrêt du découpage.
Nombre maximum de sous-rasters (750) atteint. Arrêt du découpage.


In [10]:


def count_files_in_directory(directory):
    """
    Compte le nombre de fichiers dans un dossier.

    Args:
        directory (str): Chemin du dossier.

    Returns:
        int: Nombre de fichiers dans le dossier.
    """
    try:
        # Lister les éléments dans le dossier
        items = os.listdir(directory)
        # Filtrer pour ne garder que les fichiers
        files = [item for item in items if os.path.isfile(os.path.join(directory, item))]
        return len(files)
    except FileNotFoundError:
        print(f"Le dossier '{directory}' n'existe pas.")
        return 0
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")
        return 0

# Exemple d'utilisation
directory_path = "/content/drive/MyDrive/Colab Notebooks/datasets/data_hackaton/data/raster"
file_count = count_files_in_directory(directory_path)
print(f"Nombre de fichiers dans le dossier : {file_count}")


Nombre de fichiers dans le dossier : 1483
